In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
import random

from algorithms.feature_extraction_loading import FeatureDataset, extract_diffusion_features, feature_collate_fn, concatenate_video_features

from algorithms.heatmap_generator import HeatmapGenerator
from algorithms.zero_shot_tracker import ZeroShotTracker

heatmap_generator = HeatmapGenerator()
zero_shot_tracker = ZeroShotTracker()

In [ ]:
#extract_diffusion_features(input_dataset_paths={'davis': '../tapvid_davis/tapvid_davis.pkl'}, diffusion_model_path='../text-to-video-ms-1.7b/')

In [ ]:
feature_dataset = FeatureDataset()
feature_loader = DataLoader(feature_dataset, batch_size=1, shuffle=True, collate_fn=feature_collate_fn)

for batch in feature_loader:
    for sample in batch:
        video_features = concatenate_video_features({'up_block': sample['features']['up_block'][0:3]})

        idx = random.randint(0, len(sample['query_points'][0]))
        query_point = sample['query_points'][0][idx]

        print(query_point)

        video_features = video_features.permute(0, 2, 3, 1).float()
        heatmaps = heatmap_generator.generate(video_features, (query_point[1], query_point[2], int(query_point[0])))
        tracks = zero_shot_tracker.track(heatmaps)

        zero_shot_tracker.place_marker_in_frames(sample['video'].squeeze(), tracks)
        heatmap_generator.safe_heatmap_as_gif(heatmaps)

        input("Press Enter to continue...")